In [28]:
# 필요한 라이브러리들을 로드하자
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
import undetected_chromedriver as uc

In [29]:
# 크롬 웹드라이버를 실행해서 로그인하는 과정을 함수로 나타낸거

def init_driver():
    driver = uc.Chrome()
    driver.get('https://velog.io')
    return driver
# 로그인 버튼을 눌러주고 로그인이 되서 다시 velog로 돌아올때까지 60초를 기다린다.
def do_login(driver):
    driver.find_element(By.XPATH,'//button[text()="로그인"]').click()
    WebDriverWait(driver, 60).until(
        EC.presence_of_element_located(
            (By.XPATH, '//button[text()="새 글 작성"]')
        )
    )

In [30]:
if  __name__  ==  "__main__" :
    driver = init_driver()
    do_login(driver)

In [31]:
# G10 ESI 데이터가 있는 곳을 웹드라이버로 실행시켜보자
url = 'https://en.macromicro.me/collections/25/cn-industry-relative/232/cn-pmi-caixin'  # 원하는 웹사이트 주소로 변경하세요
driver.get(url)

In [34]:
# 로그인 창이 뜰거임. 로그인 하기위한 첫 번째 과정
element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//a[@href="/login?next=%2Fcharts%2F45866%2Fglobal-citi-surprise-index"]'))
)

# 찾은 요소를 클릭합니다.
element.click()

TimeoutException: Message: 
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0069A813+48355]
	(No symbol) [0x0062C4B1]
	(No symbol) [0x00535358]
	(No symbol) [0x005609A5]
	(No symbol) [0x00560B3B]
	(No symbol) [0x0058E232]
	(No symbol) [0x0057A784]
	(No symbol) [0x0058C922]
	(No symbol) [0x0057A536]
	(No symbol) [0x005582DC]
	(No symbol) [0x005593DD]
	GetHandleVerifier [0x008FAABD+2539405]
	GetHandleVerifier [0x0093A78F+2800735]
	GetHandleVerifier [0x0093456C+2775612]
	GetHandleVerifier [0x007251E0+616112]
	(No symbol) [0x00635F8C]
	(No symbol) [0x00632328]
	(No symbol) [0x0063240B]
	(No symbol) [0x00624FF7]
	BaseThreadInitThunk [0x75447D59+25]
	RtlInitializeExceptionChain [0x774DB74B+107]
	RtlClearBits [0x774DB6CF+191]


In [ ]:
# 로그인 하기위한 두 번째 과정
element2 = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//a[@href="/login/google"]'))
)

# 찾은 요소를 클릭합니다.
element2.click()

In [32]:
element = driver.find_element(By.CSS_SELECTOR, f'.highcharts-legend-item.highcharts-line-series.highcharts-color-undefined.highcharts-series-1')
element.click()

In [35]:
# 마우스를 움직이고 클릭하기 위한 액션을 정의해줌
actions = ActionChains(driver)

In [51]:
# 내가 수동으로 찾은 초기 커서 위치로서 적합한 커서 위치
element3 = driver.find_element(By.CSS_SELECTOR, '.highcharts-axis.highcharts-yaxis')
# 찾은 요소를 클릭합니다.
actions.click(element3).perform()   # 우클릭 하고 싶으면 actions.context_click(element3).perform()

In [52]:
# 첫 데이터가 나타나는 곳으로 커서를 이동후 클릭(내가 수동으로 찾음)
actions.move_by_offset(30, 0).click().perform()

In [53]:
# 커서를 하나씩 이동 시키면서 커서를 갖다 댈때만 뜨는 데이터 값들을 크롤링 해보자

data_list = []

for i in range(950):

    # 커서를 한 칸씩 옮겨보자
    actions.move_by_offset(1, 0).click().perform()

    # 페이지의 HTML을 가져옴
    html = driver.page_source
    # BeautifulSoup 객체를 생성
    soup = bs(html, 'html.parser')
    # 클래스가 "chart-wrapper"인 요소를 찾아서 해당하는 것을 data에 저장
    wrapper = soup.find('g', attrs={'class': 'highcharts-label highcharts-tooltip highcharts-color-undefined'})
    data = wrapper.prettify()

    # 데이터에서 요일, 월, 일, 연도 추출
    date_start_index = data.find("<tspan style=\"font-size: 10px;\">") + len("<tspan style=\"font-size: 10px;\">")
    date_end_index = data.find("</tspan>")
    date = data[date_start_index:date_end_index].strip()
    # 날짜 잘 나오나 함 찍어보자
    print(date)

    # 데이터에서 G10 ESI 값을 추출
    number_start_index = data.find("<tspan style=\"font-weight: bold;\">") + len("<tspan style=\"font-weight: bold;\">")
    number_end_index = data.find("</tspan>", number_start_index)
    number = data[number_start_index:number_end_index].strip()
    # ESI값 잘 나오나 함 찍어보자
    print(number)

    # 추출한 정보를 리스트에 형태로 하나씩 추가
    data_list.append({'date': date, 'number': number})

# 커서를 하나씩 이동 시켜서 하나씩 크롤링 하는 작업을 끝냈으면 해당 데이터를 데이터 프레임으로 저장
df = pd.DataFrame(data_list)

September 2005
50.90
September 2005
50.90
September 2005
50.90
September 2005
50.90
September 2005
50.90
September 2005
50.90
September 2005
50.90
October 2005
50.10
October 2005
50.10
October 2005
50.10
October 2005
50.10
October 2005
50.10
November 2005
49.80
November 2005
49.80
November 2005
49.80
November 2005
49.80
December 2005
50.10
December 2005
50.10
December 2005
50.10
December 2005
50.10
January 2006
50.20
January 2006
50.20
January 2006
50.20
January 2006
50.20
January 2006
50.20
February 2006
50.70
February 2006
50.70
February 2006
50.70
February 2006
50.70
March 2006
51.00
March 2006
51.00
March 2006
51.00
March 2006
51.00
March 2006
51.00
April 2006
52.70
April 2006
52.70
April 2006
52.70
April 2006
52.70
May 2006
52.80
May 2006
52.80
May 2006
52.80
May 2006
52.80
May 2006
52.80
June 2006
52.90
June 2006
52.90
June 2006
52.90
June 2006
52.90
July 2006
53.00
July 2006
53.00
July 2006
53.00
July 2006
53.00
August 2006
52.60
August 2006
52.60
August 2006
52.60
August 2006
5

In [55]:
# 생성한 데이터 프레임에서 중복되는 값은 삭제
df_droped = df.drop_duplicates(subset='date')

# 그리고 중복되는 데이터를 제거한 완성된 데이터를 csv의 파일형태로 저장해둔다
df_droped.to_csv('china_pmi_lag.csv', index=False)

In [59]:
pd.read_csv('china_pmi_lag.csv')

,date,number
0,September 2005,50.9
1,October 2005,50.1
2,November 2005,49.8
3,December 2005,50.1
4,January 2006,50.2
...,...,...
208,January 2023,49.2
209,February 2023,51.6
210,March 2023,50.0
211,April 2023,49.5
